In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time
import talib as ta

In [ ]:
url = 'https://indodax.com'

In [ ]:
def get_pairs() :
    return requests.get(url + '/api/pairs').json()

In [ ]:
def get_ticker(pairid, timeframe, timefrom, timeto) :
    return requests.get(url + '/tradingview/history_v2?from=' + str(timefrom) + '&symbol=' + pairid +'&tf=' + timeframe + '&to=' + str(timeto)).json()

In [ ]:
def show_chart(df, tickername) :
    # Membuat figure dan dua subplot yang berbagi sumbu x
    fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(14, 10))

    # Plot pertama: Pergerakan Harga
    ax1.plot(df['Date'], df['Close'], label=f'{tickername}/BTC', color='blue')
    ax1.set_title(f'Pergerakan Harga {tickername} terhadap BTC')
    ax1.set_ylabel('Harga (BTC)')
    ax1.legend()
    ax1.grid(True)

    # Plot kedua: RSI
    ax2.plot(df['Date'], df['RSI_14'], label=f'RSI {tickername}/BTC', color='orange')
    ax2.axhline(70, linestyle='--', alpha=0.5, color='red')  # Garis overbought
    ax2.axhline(30, linestyle='--', alpha=0.5, color='green')  # Garis oversold
    ax2.set_title(f'Grafik RSI {tickername} terhadap BTC')
    ax2.set_xlabel('Tanggal')
    ax2.set_ylabel('RSI')
    ax2.legend()
    ax2.grid(True)

    ax3.plot(df['Date'], df['Stoch_%K'], label='%K', color='blue')
    ax3.plot(df['Date'], df['Stoch_%D'], label='%D', color='orange')
    ax3.axhline(80, linestyle='--', alpha=0.5, color='red')  # Garis overbought
    ax3.axhline(20, linestyle='--', alpha=0.5, color='green')  # Garis oversold
    ax3.set_title(f'Grafik Stochastic {tickername} terhadap BTC')
    ax3.set_xlabel('Tanggal')
    ax3.set_ylabel('Nilai')
    ax3.legend()
    ax3.grid(True)

    # Menyesuaikan tata letak agar tidak saling tumpang tindih
    plt.tight_layout()
    plt.show()

In [ ]:
def get_history_btc_pair(tickername, timeframe, timefrom, timeto, dfbtcidr) :
    ticker_name_low = tickername.lower()
    ticker_name = get_ticker(tickername + 'IDR', timeframe, timefrom, timeto)
    df = pd.DataFrame(ticker_name)
    df['Date'] = pd.to_datetime(df['Time'], unit='s')
    df = df.sort_values(by='Date')

    df_merged = pd.merge(df, dfbtcidr, on='Date', suffixes=('_'+ticker_name_low, '_btc'))
    df_merged['Open_' + ticker_name_low +'_btc'] = df_merged['Open_' + ticker_name_low] / df_merged['Open_btc']
    df_merged['High_' + ticker_name_low +'_btc'] = df_merged['High_' + ticker_name_low] / df_merged['High_btc']
    df_merged['Low_' + ticker_name_low + '_btc'] = df_merged['Low_' + ticker_name_low] / df_merged['Low_btc']
    df_merged['Close_'+ ticker_name_low +'_btc'] = df_merged['Close_' + ticker_name_low] / df_merged['Close_btc']

    df_final = df_merged[['Date', 'Open_' + ticker_name_low +'_btc', 'High_' + ticker_name_low +'_btc', 'Low_' + ticker_name_low + '_btc', 'Close_' + ticker_name_low +'_btc']].copy()
    df_final.rename(columns={
        'Open_' + ticker_name_low +'_btc': 'Open',
        'High_' + ticker_name_low +'_btc': 'High',
        'Low_' + ticker_name_low + '_btc': 'Low',
        'Close_' + ticker_name_low +'_btc': 'Close'
    }, inplace=True)

    df_final['RSI_14'] = ta.RSI(df_final['Close'], timeperiod=14)

    slowk, slowd = ta.STOCH(df['High'], df['Low'], df['Close'],
                           fastk_period=5,
                           slowk_period=3,
                           slowd_period=3)
    df_final['Stoch_%K'] = slowk
    df_final['Stoch_%D'] = slowd

    return df_final

In [ ]:
pairs = get_pairs()
symbols = [item['symbol'] for item in pairs if 'symbol' in item and item['symbol'].endswith('IDR') and item['symbol'] != 'BTCIDR']
print(symbols)

In [ ]:
now = datetime.now()
past = now - timedelta(days=5*365)
timeFrom = int(time.mktime(past.timetuple()))
timeTo = int(time.mktime(now.timetuple()))
tf = '1D'

In [ ]:
btcIdr = get_ticker('BTCIDR', tf, timeFrom, timeTo)
dfBtcIdr = pd.DataFrame(btcIdr)
dfBtcIdr['Date'] = pd.to_datetime(dfBtcIdr['Time'], unit='s')
dfBtcIdr = dfBtcIdr.sort_values(by='Date')

In [ ]:
def show_chart_rsi(df) :
    plt.figure(figsize=(10, 6))
    plt.scatter(df['Label'], df['RSI_14'], color='blue')
    plt.axhline(70, linestyle='--', alpha=0.5, color='red', label='Overbought (70)')
    plt.axhline(30, linestyle='--', alpha=0.5, color='green', label='Oversold (30)')
    plt.title('Nilai RSI Terbaru')
    plt.xlabel('Ticker')
    plt.ylabel('RSI')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
tickers = ['FARTCOIN', 'MOODENG', 'SPX', 'ZEREBRO', 'WIF', 'ACTSOL', 'LADYS', 'ANDY', 'APU', 'BOME', 'BONE', 'BRETT', 'CAT', 'CATI', 'CHILLGUY', 'DEGEN', 'DOGE', 'DOGS', 'FLOKI', 'FWOG', 'GIGA', 'GOAT', 'MYRO', 'MEW', 'MOG', 'PEPE', 'NPC', 'PEOPLE', 'PONKE', 'POPCAT', 'SHIB', 'SLERF', 'SUNDOG', 'TEN', 'TURBO', 'PNUT']

tickers_error = ['NEIRO', 'VIRTUAL']

rsi_list = []

for i, ticker in enumerate(tickers) :
    df = get_history_btc_pair(ticker, tf, timeFrom, timeTo, dfBtcIdr)
    show_chart(df, ticker)
    latest_rsi = df['RSI_14'].iloc[-1]
    rsi_list.append({'Label': ticker, 'RSI_14': latest_rsi})

rsi_df = pd.DataFrame(rsi_list)
show_chart_rsi(rsi_df)

In [ ]:
tickers = [ 'AERO', 'ALTLAYER', 'API3', 'ARB', 'ARKM', 'ATH', 'AVAX', 'BGB', 'BNB', 'CAKE', 'CTC', 'CRV', 'CTK', 'EIGEN', 'ENA', 'ENS', 'ETH', 'FET', 'FIL', 'FLUX', 'FTM', 'GALA', 'GMT', 'GRASS', 'GRT', 'HBAR', 'HIFI', 'HNT', 'IMX', 'IO', 'ISLM', 'JASMY', 'JUP', 'JTO', 'KMNO', 'LDO', 'LPT', 'LQTY', 'MBOX', 'ME', 'METIS', 'MKR', 'MNT', 'MOVE', 'NEAR', 'OM', 'ONDO', 'PAXG', 'PENDLE', 'PHA', 'PYTH', 'POL', 'RAY', 'REZ', 'RENDER', 'SOL', 'STORJ', 'SUI', 'TON', 'TRX', 'UNI', 'WORMHOLE', 'XRP', 'ZKJ']

tickers_error = ['AAVE', 'FORM', 'ID', 'XDC']

rsi_list = []

for i, ticker in enumerate(tickers) :
    df = get_history_btc_pair(ticker, tf, timeFrom, timeTo, dfBtcIdr)
    show_chart(df, ticker)
    latest_rsi = df['RSI_14'].iloc[-1]
    rsi_list.append({'Label': ticker, 'RSI_14': latest_rsi})

rsi_df = pd.DataFrame(rsi_list)
show_chart_rsi(rsi_df)
